In [7]:
import os
import re
import cantera as ct

#directory = '/work/westgroup/nora/Code/projects/halogens/refrigerants/patterns_in_blends/blends_of_2/'

def convert_files():
    """convert chemkin files to .cti files. Can be used when there are 'unmarked' duplicates
    """
    
    converter = \
    '''

import cantera as ct
import numpy as np
import pandas as pd
import os 
import re
from subprocess import getoutput
import sys
import csv


print("Running Cantera Version: " + str(ct.__version__))





############### copies chemkin files to dups folder #############################


directory ='.'


#os.system('source activate ct_env') #if on local, this is cantera 2.6 beta
os.system('source activate cantera_env') #if on discovery, will be cantera 2.5


#copy folders so i dont screw up the original, and change into the new directory with copies
os.makedirs('copies', exist_ok=True)

#copy chem.inp file into dups folder, and will then convert this copy into .cti
os.command = f'scp chem.inp copies/copy_chem.inp'
os.system(os.command)
os.system('scp tran.dat copies/tran.dat')

#now look in the dups folder
os.chdir('./copies')









############################# converts the dup_chem.inp files to .cti files #############################

x = 0
while x == 0: 
#this is a string of the output when I try to convert this file to a cti file. Will probably produce an error
    output = getoutput( f'ck2cti --input=copy_chem.inp --transport=tran.dat') 
    #if command passed without an error
    if re.search('PASSED',output):
        print('**************************command passed, converting to cti***************************')
        x += 1 
    #if command generated the Duplicate error
    else:
        print('*******************************needs some work****************************************')
        if re.search('Encountered\sunmarked\sduplicate\sreaction',output):
            match = re.search('See\slines\s([0-9]+)\sand\s([0-9]+)\sof\sthe\sinput\sfile',output)
            #capture the line numbers with the duplicate reactions
            line_numbers = [int(match.group(1)), int(match.group(2))]
            print(f'Unmarked duplicates on lines {match.group(1)} and {match.group(2)}')
            print('Editing chemkin file to allow conversion to .cti')
            #write the lines of the chemkin input file to a list so that I can insert the "DUPLICATE" statement
            with open('copy_chem.inp','r') as f:
                data = f.readlines()
                print(data[line_numbers[0]-1], data[line_numbers[1]-1])

            #start editing the .inp file below

            #'adjustments' will make sure that, even when I add an element in 'data', my index will still be correct
            adjustments = [0,1]
            for i,adjust in zip(line_numbers,adjustments): 
                start = i+adjust-1
                count = 0 
                while count == 0: 
                    #if you don't see a blank line after the duplicated reaction line, keep going until you do
                    if not re.search('^\\n', data[start]): 
                        print('no match')
                        print(start)
                        print(data[start])
                        start += 1
                    #when we get to the blank line after the reaction block, insert "DUPLICATE" and stop the loop for this line number
                    else: 
                        print('there is a match')
                        data.insert(start,'DUPLICATE')
                        count = 1 
            #now overwrite the input file with the change 
            with open(f'copy_chem.inp','w+') as f: 
                for l in data: 
                    f.write(l)
                    x==0

        #if the command generated an error that is not the Duplicate error
        else:
            #if code ever gets to here, just cry
            print('There is another error, see Output')
            print(output)
            x += 1
            
            
'''   
    directory = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/Nora/2_BTP/FFCM_seed/CH4_with_seed/chemkin'
    os.chdir(directory)


    with open(os.path.join(directory,'converter.py'),'w') as f:
        for l in converter:
            f.write(l)
    print('rewrote it!')

#list_of_blends = [file for file in os.listdir(directory) if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]
#list_of_blends = [ 'C2H5F', 'C2H6', 'CH2F2', 'CH2FCH2F', 'CH2FCHF2', 'CH3CF3', 'CH3CHF2', 'CH3F', 'CH4']

# for blend in list_of_blends: 
#     d = os.path.join(directory, f'{blend}/chemkin')
#     with open(os.path.join(d,'converter.py'),'w') as f:
#         for l in converter:
#             f.write(l)
#     print('rewrote it!')



# directory = '/work/westgroup/nora/Code/projects/halogens/refrigerants/blends'

# list_of_blends = [file for file in os.listdir(directory) if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]
# #list_of_blends = ['CH3F_CH3CF3']

convert_files()


            

rewrote it!
